In [1]:
import numpy as np

In [27]:
class op:
    def __init__(self):
        self.parents = []
        self.children = []
    
    def name(self):
        return self._name
    
    def add_parent(self, parent):
        self.parents.append(parent)
                
    def compute_children(self):
        try:
            return self.children_values
        except AttributeError:
            self.children_values = [c.compute() for c in self.children]
            return self.children_values
        
    def compute(self):
        try:
            return self.value
        except AttributeError:
            self.value = self.compute_no_cache()
            return self.value
        
    def differentiate_parents(self):
        try:
            return self.parent_deriv
        except AttributeError:
            if len(self.parents) == 0:
                self.parent_deriv = 1.0
            else:
                self.parent_deriv = np.sum([p.differentiate(self) for p in self.parents])
            return self.parent_deriv 

    def symbolic_differentiate_parents(self):
        try:
            return self.sym_parent_deriv
        except AttributeError:
            if len(self.parents) == 0:
                self.sym_parent_deriv = "1"
            else:
                self.sym_parent_deriv = " + ".join([p.symbolic_differentiate(self) for p in self.parents])
                self.sym_parent_deriv = "(" + self.sym_parent_deriv + ")"
            return self.sym_parent_deriv 
    
class add(op):
    def __init__(self, children):
        super().__init__()
        self.children = children
        self._name = " + ".join([c.name() for c in children])
        for c in children:
            c.add_parent(self)
                
    def compute_no_cache(self):
        return np.sum([self.compute_children()])
        
    def differentiate(self, child):
        return self.differentiate_parents()
    
    def symbolic_differentiate(self, child):
        return self.symbolic_differentiate_parents()



class prod(op):
    def __init__(self, children):
        super().__init__()
        self.children = children
        self._name = " * ".join(["(" + c.name() + ")" for c in children])
        for c in self.children:
            c.add_parent(self)
                
    def compute_no_cache(self):
        return np.prod(self.compute_children())
        
    def differentiate(self, child):
        p = 1.0
        removed = False
        for (c,v) in zip(self.children, self.compute_children()):
            if removed or c.name() != child.name():
                p *= v
            else:
                removed = True
        return self.differentiate_parents() * p
    
    def symbolic_differentiate(self, child):
        p = []
        removed = False
        for c in self.children:
            if removed or c.name() != child.name():
                p.append("(" + c.name() + ")")
            else:
                removed = True
        return self.symbolic_differentiate_parents() + " * " + " * ".join(p)

    
class inv(op):
    def __init__(self, child):
        super().__init__()
        self.children = [child]
        self._name = "1.0/(" + child.name() + ")"
        for c in self.children:
            c.add_parent(self)
                
    def compute_no_cache(self):
        return 1.0 / self.compute_children()[0]
        
    def differentiate(self, child):
        return -self.differentiate_parents() / (self.compute_children()[0] ** 2.0)

    def symbolic_differentiate(self, child):
        return "-" + self.symbolic_differentiate_parents() + " / (" + self.children[0].name() + ")**2.0"

class exp(op):
    def __init__(self, child):
        super().__init__()
        self.children = [child]
        self._name = "np.exp(" + child.name() + ")"
        for c in self.children:
            c.add_parent(self)
                
    def compute_no_cache(self):
        return np.exp(self.compute_children()[0])
        
    def differentiate(self, child):
        return self.differentiate_parents() * np.exp(self.compute_children()[0])

    def symbolic_differentiate(self, child):
        return  self.symbolic_differentiate_parents() + " * np.exp(" + self.children[0].name() + ")"




    
class variable(op):
    def __init__(self, name):
        super().__init__()
        self._name = name
    
    def set_value(self, value):
        self.value = value
        return self
    
    def compute_no_cache(self):
        return self.value
    
    def differentiate(self):
        return self.differentiate_parents()
    
    def symbolic_differentiate(self):
        return self.symbolic_differentiate_parents()


        
    

In [30]:
xval = 3.
yval = 1.0
x = variable("x").set_value(xval)
y = variable("y").set_value(yval)
a = add([x,y])
# r = prod([prod([add([x,a]),inv(prod([a, a]))]), exp(prod([add([a,a]),inv(prod([a, a]))]))])
r = inv(exp(add([a,y])))
print(a.name())
print(r.name())
print(r.compute())
print(x.differentiate())
print(y.differentiate())
print(x.symbolic_differentiate())
print(y.symbolic_differentiate())

x + y
1.0/(np.exp(x + y + y))
0.00673794699909
-0.00673794699909
-0.0134758939982
((((-1 / (np.exp(x + y + y))**2.0) * np.exp(x + y + y))))
((((-1 / (np.exp(x + y + y))**2.0) * np.exp(x + y + y))) + ((-1 / (np.exp(x + y + y))**2.0) * np.exp(x + y + y)))


In [31]:
def f(x,y):
    return 1.0/(np.exp(x + y + y))
def dfdx(x,y):
    return ((((-1 / (np.exp(x + y + y))**2.0) * np.exp(x + y + y))))
def dfdy(x,y):
    return ((((-1 / (np.exp(x + y + y))**2.0) * np.exp(x + y + y))) + ((-1 / (np.exp(x + y + y))**2.0) * np.exp(x + y + y)))


print(f(xval,yval))
h = 0.00000001
print((f(xval + h, yval) - f(xval, yval) ) / h)
print((f(xval, yval + h) - f(xval, yval) ) / h)

print(dfdx(xval, yval))
print(dfdy(xval, yval))

0.00673794699909
-0.00673794691916
-0.0134758938383
-0.00673794699909
-0.0134758939982
